In [1]:
from openai import OpenAI
import numpy as np

In [2]:
def cosine_similarity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

In [ ]:
client = OpenAI(api_key="<API_KEY>")

In [14]:
with open('messi.txt', 'r') as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [15]:
CHUNK_SIZE = 64
chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

In [16]:
embedded_chunks = [
    (chunk, client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    ).data[0].embedding)
    for chunk in chunks
]

In [17]:
question = "Where was Messi born?"
embedded_question = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding

In [18]:
embedded_chunks_similarities = list(map(lambda x: (x[0], x[1], cosine_similarity(x[1], embedded_question)), embedded_chunks))

In [23]:
top_5_chunks = sorted(embedded_chunks_similarities,
                      key=lambda x: x[2], reverse=True)[:5]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {" ".join([f"{i+1}. {chunk[0]}" for i, chunk in enumerate(top_5_chunks)])}"
        }
    ]
)

In [24]:
print(completion.choices[0].message.content)

Lionel Messi was born in Rosario, Argentina.
